In [412]:
!pwd


/home/notc/crypto-data


In [291]:
import requests
import pandas as pd 
import warnings
import datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas_datareader.data as web
from plotly.subplots import make_subplots
import re
warnings.filterwarnings('ignore')

In [292]:
START_DATE = "2019-07-01"
RSI_TIME_WINDOW = 7

In [334]:
# list of urls of historical crypto data and names

urls = ["https://www.cryptodatadownload.com/cdd/Binance_DOGEUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_XRPUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_SOLUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_MATICUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_TRXUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_DOTUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_AVAXUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_LINKUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_UNIUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_XMRUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_ETCUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_XLMUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_ICPUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_BCHUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_1INCHUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_AAVEUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_ADAUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_ALGOUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_ALPACAUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_SUSHIUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_FILUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_HBARUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_QNTUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_FTMUSDT_d.csv"
       ]
    
crypto_names = [
        "TUSD (TUSD)",
        "1inch (1INCH)",
        "Aave (AAVE)",
        "Cardano (ADA)",
        "Algorand (ALGO)",
        "Alpaca Finance (ALPACA)",
        "USD Coin (USDC)",
        "Binance Coin (BNB)",
        "Dogecoin (DOGE)",
        "XRP (XRP)",
        "Solana (SOL)",
        "Polygon (MATIC)",
        "Litecoin (LTC)",
        "TRON (TRX)",
        "Dogecoin (DOGE)",
        "SushiSwap (SUSHI)",
        "Avalanche (AVAX)",
        "USDT/DAI (USDTDAI)",
        "Chainlink (LINK)",
        "Uniswap (UNI)",
        "XMR (XMR)",
        "ETC (ETC)",
        "XLM (XLM)",
        "Internet Computer (ICP)",
        "Bitcoin Cash (BCH)"
]

In [336]:

def df_loader(urls, start_date="2019-07-01"):
    """
    Loads cryptocurrency price data from a list of URLs and returns a DataFrame and a list of filenames.

    Args:
    urls: A list of URLs pointing to CSV files containing cryptocurrency price data.
    start_date: A string representing the start date of the data.

    Returns:
    A tuple of a DataFrame and a list of filenames.
    """

    filenames = []
    all_df = pd.DataFrame()
    for url in urls:
        try:
            req = requests.get(url, verify=False)
            url_content = req.content
            pattern = r'_[A-Z]+(?=USDT)'
            match = re.search(pattern, url)
            if match:
                ticker_symbol = match.group(0)[1:]
                filenames.append(ticker_symbol)
                csv_file = open(ticker_symbol + "USDT_d.csv", 'wb')
                csv_file.write(url_content)
                csv_file.close()

                df = pd.read_csv(ticker_symbol + "USDT_d.csv", header=1, parse_dates=["Date"])
                df = df[df["Date"] > start_date]
                df.index = df["Date"]
                df.drop(labels=[df.columns[0], df.columns[1], df.columns[7]], axis=1, inplace=True)
                # df.drop(labels=["Volume 1INCH"], axis=1, inplace=True)
                all_df = pd.concat([all_df, df], ignore_index=False)
        except FileNotFoundError:
            pass
    return all_df, filenames


In [337]:
all_df, filenames = df_loader(urls, start_date=START_DATE)
crypto_df = []


In [340]:
def computeRSI(data, time_window):
    """
    Computes the Relative Strength Index (RSI) for a given time series.

    Args:
    data: A pandas DataFrame containing the time series data.
    time_window: The number of periods to use for the RSI calculation.

    Returns:
    A pandas Series containing the RSI values.
    """

    diff = data.diff(1).dropna()
    up_chg = 0 * diff
    down_chg = 0 * diff
    up_chg[diff > 0] = diff[ diff>0 ]
    down_chg[diff < 0] = diff[ diff < 0 ]
    up_chg_avg = up_chg.ewm(com=time_window-1, min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1, min_periods=time_window).mean()
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi


In [341]:
def computeMovingAverage(data, time_window):
    """
    Computes the moving average for a given time series.

    Args:
    data: A pandas DataFrame containing the time series data.
    time_window: The number of periods to use for the moving average calculation.

    Returns:
    A pandas Series containing the moving average values.
    """

    moving_average = data.rolling(window=time_window).mean()
    return moving_average


In [375]:
def computeBollingerBands(data, time_window, num_stds):
    """
    Computes Bollinger Bands for a given time series.

    Args:
    data: A pandas DataFrame containing the time series data.
    time_window: The number of periods to use for the moving average calculation.
    num_stds: The number of standard deviations to use for the Bollinger Bands calculation.

    Returns:
    A pandas DataFrame containing the Bollinger Bands values.
    """

    moving_average = data.rolling(window=time_window).mean()
    standard_deviation = data.rolling(window=time_window).std()
    upper_band = moving_average + num_stds * standard_deviation
    lower_band = moving_average - num_stds * standard_deviation

    return pd.DataFrame({"Moving Average": moving_average, "Upper Band": upper_band, "Lower Band": lower_band})


In [378]:
for symbol in all_df["Symbol"].unique():
    symbol_df = all_df[all_df["Symbol"] >= start_date]
    symbol_df = all_df[all_df["Symbol"] == symbol]
    close = symbol_df["Close"]
    rsi = computeRSI(close, 7)
    
    symbol_df["rsi_14"] = rsi
    symbol_df["overbought_threshold"] = 70  
    symbol_df["oversold_threshold"] = 30
    
    # 50 period simple moving average
    ma50 = close.rolling(50).mean()
    symbol_df["ma50"] = ma50
    
    # 20 period Bollinger Bands    
    std20 = close.rolling(20).std()
    upper_bb = ma50 + 2*std20
    lower_bb = ma50 - 2*std20
    
    symbol_df["upper_bb"] = upper_bb
    symbol_df["lower_bb"] = lower_bb


In [410]:
symbol_df

,Symbol,Open,High,Low,Close,Volume USDT,tradecount,rsi_14,overbought_threshold,oversold_threshold,ma50,upper_bb,lower_bb
Date,,,,,,,,,,,,,
2023-05-16,FTMUSDT,0.38260,0.38600,0.37340,0.37940,1.834623e+07,54586,NaN,70,30,NaN,NaN,NaN
2023-05-15,FTMUSDT,0.36990,0.39440,0.36230,0.38260,2.842510e+07,76547,NaN,70,30,NaN,NaN,NaN
2023-05-14,FTMUSDT,0.36430,0.37590,0.35890,0.37000,1.411766e+07,40585,NaN,70,30,NaN,NaN,NaN
2023-05-13,FTMUSDT,0.37230,0.37300,0.36220,0.36420,1.495509e+07,39553,NaN,70,30,NaN,NaN,NaN
2023-05-12,FTMUSDT,0.36810,0.37380,0.35160,0.37230,3.789862e+07,100864,NaN,70,30,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-06,FTMUSDT,0.02751,0.02788,0.02568,0.02576,8.623445e+05,5586,66.324359,70,30,0.020582,0.024599,0.016565
2019-07-05,FTMUSDT,0.02735,0.02947,0.02620,0.02751,1.341997e+06,7867,73.397400,70,30,0.020810,0.025418,0.016202
2019-07-04,FTMUSDT,0.03037,0.03268,0.02585,0.02735,3.384859e+06,12974,71.789062,70,30,0.021019,0.026059,0.015980
